### Spring cache

Aby przyśpieszyć działanie aplikacji, i zredukować ilość zapytań tych samych zapytań do bazy danych, możemy przechowywać wyniki zapytań w pamięci podręcznej, w pamięci RAM lub na dysku. 

Aby móc cache-ować wyniki zapytań w Spring Boot należy dodać adnotację @EnableCaching w pliku ...Application oraz adnotację @Cacheable do metody w serwisie. Adnotacja ma kilka parametrów. My dodamy cacheNames aby skonfigurować nazwę cache.

In [ ]:
## in ...application
@SpringBootApplication
@EnabledCaching
public class ....() {
    ...
}

## in service
@Cacheable(cacheNames = "PostsWithComments")
public List<Post> getPostsWithComments(int page, Sort.Direction sort) {
    List<Post> allPosts = postRepository.findAllPosts(
            PageRequest.of(page, PAGE_SIZE,
                    Sort.by(sort, "id")
            )
    );
    List<Long> ids = allPosts.stream()
            .map(Post::getId)
            .collect(Collectors.toList());
    List<Comment> comments = commentRepository.findAllByPostIdIn(ids);
    allPosts.forEach(post -> post.setComment(extractComments(comments, post.getId())));
    return allPosts;
}

Cache już działa, ale nie mamy łatwej możliwości zarządzanie wpisami w tym cashu (np. usunąć cache po upływie jakiegoś czasu).
Aby nie musieć robić tego "ręcznie" możemy użyć zewnętrznych Providerów. Wystarczy dodać odpowiednią konfigurację.

W tym przykładzie skorzystamy z Hazelcast.

EHCACHE. (NIE DZIAŁA! NIE JEST JUŻ W CacheConfiguations.java!!!!!!! Czyżby Spring zrezygnował z tego providera?)

Najpierw dodajemy do build.gradle nowe zależności.

In [ ]:
    implementation 'org.ehcache:ehcache:3.10.8'
    implementation 'javax.cache:cache-api:1.1.0'
    
    ## for hazelcast 
    implementation 'org.springframework.boot:spring-boot-starter-cache'
    implementation 'com.hazelcast:hazelcast-all:4.2.4'